In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import math
from plotly.subplots import make_subplots
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential, Model,load_model
from keras.applications import VGG19, VGG16, ResNet50
from tensorflow.keras.applications import EfficientNetB0
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
import cv2
from keras.initializers import glorot_uniform

import warnings
warnings.filterwarnings("ignore")




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input/landmark-recognition-2021/train'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#os.listdir('../input/landmark-recognition-2021/')

In [ ]:
train_image_dir = '../input/landmark-recognition-2021/train'
test_image_dir = '../input/landmark-recognition-2021/test'
train = pd.read_csv("../input/landmark-recognition-2021/train.csv")
sample_submission = pd.read_csv('../input/landmark-recognition-2021/sample_submission.csv')

In [ ]:
test = pd.DataFrame()
test['id'] = sample_submission['id']

In [ ]:
test

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train['landmark_id'].value_counts()

In [ ]:
train['id_counts'] = train['landmark_id'].value_counts()[train['landmark_id'].values].values
id_map = train.sort_values(by='id_counts',ascending=False).landmark_id.drop_duplicates().reset_index(drop=True)
#id_map
id_dict = {id_map[x]:x for x in range(len(id_map))}
#id_dict
train['encoded_id'] = train['landmark_id'].apply(lambda x: id_dict[x])
train = train.sort_values(by = 'encoded_id', ascending=True)
train

In [ ]:
drop_df = train[train['id_counts']<5]
drop_df.shape

In [ ]:
sample_submission

In [ ]:
"""
landmark = train["landmark_id"].value_counts()
landmark = pd.DataFrame({"id":landmark.index,"frequency":landmark.values})
landmark['landmark_id'] = landmark['id'].apply(lambda x: "id_"+str(x))
landmark
"""

In [ ]:
"""
fig = px.bar(landmark.head(30),x="frequency",y="landmark_id",color="landmark_id",height=1000,title="Number of images per landmark_id (Top 30 landmark_ids)")
fig.show()
"""

In [ ]:
"""
fig = make_subplots(rows=1, cols=2)
trace1 = go.Histogram(x=landmark["frequency"],xaxis="x1",yaxis="y1")
trace2 = go.Box(y=landmark['frequency'],xaxis="x2",yaxis="y2")
layout = go.Layout(
    yaxis1=dict(
        range=[0,100],
        anchor="y1"
    ),
    yaxis2=dict(
        range=[0,100],
        anchor="y2"
    )
)

fig.add_trace(trace1, row=1,col=1)
fig.add_trace(trace2, row=1,col=2)
#fig = px.histogram(landmark, x="frequency",range_y=[0,100])
#fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.update_layout(layout)
fig.show()
"""

In [ ]:
#landmark['frequency'].describe()

In [ ]:
#landmark[landmark['frequency']>100].shape[0]

In [ ]:
def show_image(names,title=None):
    fig, axs = plt.subplots(4,4,figsize=(24,28))
    if title:
        fig.suptitle(title, fontsize = 30)
    for i, image_name in enumerate(names):
        image_path = os.path.join(train_image_dir,f"{image_name[0]}/{image_name[1]}/{image_name[2]}/{image_name}.jpg")
        image = Image.open(image_path)
        
        axs[i//4, i%4].imshow(image) 
        image.close()       
        axs[i//4, i%4].axis('off')
        
        id = train[train.id==image_name].landmark_id.values[0]
        axs[i//4, i%4].set_title(f"ID: {image_name}\nLandmark_id: {id}", fontsize="12")
    plt.show()

In [ ]:
#samples = train.sample(16).id.values
#show_image(samples,'random')

In [ ]:
#top1 = train[train.landmark_id==138982].sample(16).id.values
#show_image(top1,'138982')

In [ ]:
#top2 = train[train.landmark_id==126637].sample(16).id.values
#show_image(top2,'126637')

In [ ]:
#top3 = train[train.landmark_id==20409].sample(16).id.values
#show_image(top3,'20409')

In [ ]:
train['path'] = train['id'].apply(lambda x: os.path.join(train_image_dir,f"{x[0]}/{x[1]}/{x[2]}/{x}.jpg"))

In [ ]:
test['path'] =  test['id'].apply(lambda x: os.path.join(test_image_dir,f"{x[0]}/{x[1]}/{x[2]}/{x}.jpg"))

In [ ]:
test

In [ ]:
train

In [ ]:
train_size = len(list(train.index))-1

In [ ]:
train_size

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, path, list_IDs, data, img_size, img_channel, batch_size, augmentation=True):
        self.path = path
        self.list_IDs = list_IDs
        self.data = data
        #self.labels = labels
        self.img_size = img_size
        self.img_channel = img_channel
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.list_IDs))
        self.augmentation = augmentation
    
    def __len__(self):
        len_ = int(math.ceil(len(self.list_IDs)/batch_size))
        if len_*self.batch_size < len(self.list_IDs):
            len_ += 1
        return len_
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y
    
    def __data_generation(self, list_IDs_temp):
        batch_size_ = len(list_IDs_temp)
        if self.augmentation:
            batch_size_ = batch_size_ * 2
        X = np.zeros((batch_size_, self.img_size, self.img_size, self.img_channel))
        y = np.zeros((batch_size_,1))
        
        for i, ID in enumerate(list_IDs_temp):
            img_id = self.data.loc[ID,'id']
            img_path = self.data.loc[ID,'path']
            img = np.float32(cv2.resize(cv2.imread(img_path),(self.img_size,self.img_size))/255)
            #plt.imshow(img)
                        
            if self.augmentation:
                img1 = self.__image_augmentation(img)
                
                X[i*2, ] = img
                X[i*2+1, ] = img1
                #X[i*3+2, ] = img2
                #X[i*3+3, ] = img4
                y[i*2, ] = self.data.loc[ID, 'encoded_id']
                y[i*2+1, ] = self.data.loc[ID, 'encoded_id']
                #y[i*3+2, ] = self.data.loc[ID, 'encoded_id']
                #y[i*3+3, ] = self.data.loc[ID, 'encoded_id']
                
            else:
                X[i,] = img
                if self.path.find('train')>0:
                    y[i,] = self.data.loc[ID, 'encoded_id']
                else:
                    y[i,] = 0
            
        return X, y
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)
        #self.list_IDs = list(train_size - np.array(self.list_IDs))
        
        
    def __image_augmentation(self, image):
        
        #aug_img1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #aug_img1 = cv2.cvtColor(aug_img1, cv2.COLOR_BGR2RGB)
        
        #M2 = cv2.getRotationMatrix2D(((self.img_size-1)/2.0,(self.img_size-1)/2.0),90,1)
        #aug_img2 = cv2.warpAffine(image,M2,(self.img_size,self.img_size))
        
        #M3 = cv2.getRotationMatrix2D(((self.img_size-1)/2.0,(self.img_size-1)/2.0),315,1)
        #aug_img3 = cv2.warpAffine(image,M3,(self.img_size,self.img_size))
        
        srcTri = np.array( [[0, 0], [image.shape[1] - 1, 0], [0, image.shape[0] - 1]] ).astype(np.float32)
        dstTri = np.array( [[0, image.shape[1]*0.33], [image.shape[1]*0.85, image.shape[0]*0.25], [image.shape[1]*0.15, image.shape[0]*0.7]] ).astype(np.float32)
        M4 = cv2.getAffineTransform(srcTri,dstTri)
        aug_img4 = cv2.warpAffine(image,M4,(self.img_size,self.img_size))
        
        return aug_img4
        
    

In [ ]:
IDs_train = list(train.index)[::4]

In [ ]:
#IDs_train,IDs_val = train_test_split(list(train.index)[:50000],test_size=0.3,random_state=0)

In [ ]:
#len(IDs_train), len(IDs_val)

In [ ]:
img_size=32
img_channel=3
batch_size=256
epochs = 15
num_classes = len(train['landmark_id'].value_counts())

train_generator = DataGenerator(train_image_dir, IDs_train, train, img_size, img_channel, batch_size,augmentation=False)
#val_generator = DataGenerator(train_image_dir, IDs_val, train, img_size, img_channel, batch_size,augmentation=False)

In [ ]:
"""
conv_base = EfficientNetB0(include_top=False,weights=None,
                     input_shape=(img_size, img_size, img_channel))
conv_base.trainable = True
"""

In [ ]:
"""
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer = Adam(lr=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=['sparse_categorical_accuracy'])

model.summary()
"""

In [ ]:
def identity_block(X, f, filters, stage, block):
    """ 
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X
    
    

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [ ]:
X_input = Input([img_size,img_size, img_channel])
X = ZeroPadding2D((3, 3))(X_input)

X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

X = convolutional_block(X, f=3, filters=[64, 64, 128], stage=2, block='a', s=1)
X = identity_block(X, 3, [64, 64, 128], stage=2, block='b')
X = identity_block(X, 3, [64, 64, 128], stage=2, block='c')

X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

X = AveragePooling2D((2,2), name="max_pool")(X)

X = Flatten()(X)

X = Dense(1024, activation='relu', name='fc'+'-temporary',kernel_initializer = glorot_uniform(seed=0))(X)
X = Dense(num_classes, activation='softmax', name='fc' + str(num_classes), kernel_initializer = glorot_uniform(seed=0))(X)

model2 = Model(inputs = X_input, outputs = X, name='Simplified-ResNet50')

model2.compile(optimizer = Adam(lr=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=['sparse_categorical_accuracy'])

model2.summary()

In [ ]:

history = model2.fit_generator(generator=train_generator,
                              #validation_data=val_generator,
                              epochs = epochs, workers=32)


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
#history = model.fit_generator(generator=train_generator,
#                              epochs = epochs, workers=16)

In [ ]:
subsize_len = 1000 
encoded_results = []
results = []
scores = []
test_len = len(list(test.index))
if test_len > 1000:
    for i in range(math.ceil(len(list(test.index))/subsize_len)):
        IDs_test = list(test.index)[i*subsize_len:(i+1)*subsize_len]
        test_generator = DataGenerator(test_image_dir, IDs_test, test, img_size, img_channel, batch_size, augmentation=False)


        y_pred = model2.predict_generator(test_generator, verbose =1)

        encoded_result = [np.argmax(s) for s in y_pred]
        result = [id_map[er] for er in encoded_result]
        score = [y_pred[i][encoded_result[i]].round(2) for i in range(len(encoded_result))]


        encoded_results.extend(encoded_result)
        results.extend(result)
        scores.extend(score)

else:
    IDs_test = list(test.index)
    test_generator = DataGenerator(test_image_dir, IDs_test, test, img_size, img_channel, batch_size, augmentation=False)
    y_pred = model2.predict_generator(test_generator, verbose =1)
    encoded_result = [np.argmax(s) for s in y_pred]
    result = [id_map[er] for er in encoded_result]
    score = [y_pred[i][encoded_result[i]].round(2) for i in range(len(encoded_result))]
    encoded_results.extend(encoded_result)
    results.extend(result)
    scores.extend(score)

In [ ]:
test['landmarks'] = [str(results[i])+' '+str(scores[i]) for i in range(len(scores))]

In [ ]:
test.drop(['path'], axis=1,inplace=True)

In [ ]:
test

In [ ]:
test.to_csv('submission.csv',index=False)